In [1]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow.feather as feather
from sklearn.preprocessing import LabelEncoder

In [2]:
#Define file paths
file_path_training = 'D:\\Masters_Courses\\ELG5255_Applied_Machine_Learning\\Project_NAPS\\NAPS\\data\\train_series.parquet'
file_path_events = 'D:\\Masters_Courses\\ELG5255_Applied_Machine_Learning\\Project_NAPS\\NAPS\\data\\train_events.csv'

In [3]:
# Reading the Parquet file into a pandas DataFrame
training_df = pd.read_parquet(file_path_training)
train_events_df = pd.read_csv(file_path_events)

In [5]:
#Encoding Categorical Variables

label_encoder = LabelEncoder()
#training_df['series_id'] = label_encoder.fit_transform(training_df['series_id'])
train_events_df['series_id'] = label_encoder.fit_transform(train_events_df['series_id'])

In [8]:
train_events_df = train_events_df.drop('timestamp', axis=1)
train_events_df.dropna(subset=['step'], inplace=True) #dropping all nan values from training events

In [9]:
import dask.dataframe as dd

# Convert pandas dataframes to Dask dataframes
dask_training_df = dd.from_pandas(training_df, npartitions=10)
dask_train_events_df = dd.from_pandas(train_events_df, npartitions=10)

# Merge Dask dataframes
merged_dask_df = dd.merge(dask_training_df, dask_train_events_df, on=['series_id', 'step'], how='left')

# Compute the result to obtain a pandas dataframe
merged_df = merged_dask_df.compute()


c:\Users\User\anaconda3\lib\site-packages\dask\dataframe\multi.py:469: UserWarning: Merging dataframes with merge column data type mismatches: 
+------------------+------------+-------------+
| Merge columns    | left dtype | right dtype |
+------------------+------------+-------------+
| ('step', 'step') | uint32     | float64     |
+------------------+------------+-------------+
Cast dtypes explicitly to avoid unexpected results.
  warnings.warn(


In [13]:
# merging event information
# Merge the dataframes on series_id and step
training_merged_df = pd.merge(training_df, train_events_df, on=['series_id', 'step'], how='left')

# Display the merged dataframe
print(training_merged_df)


In [ ]:
#save pre-processed data to parquet files:
training_df.to_parquet('processed_training_data.parquet', index=False)
test_df.to_parquet('processed_test_data.parquet', index=False)